# Starbucks in Manhattan?

Our objective today is to map out and explore every Starbuck location in Manhattan. For the purposes of analysis, we 
will then use that list to compute average distance-to-Starbucks in Manhattan.

## First, some background

There are a few ways to get data off of the web: OCR, web scraping, APIs, and data stores are all valid approaches which are variously necessary for different specific problems. For our problem we will be using an API---specifically, the Yelp! API. We need a list of coordinates of chain store locations in Manhattan, and---lo and behold---Yelp! provides us with just such a list! We just have to figure out how to get it out of their hands into ours.

For brevity, we are actually skipping one of the hardest parts of a data science project: figuring out what tools to use to approach the problem. Until you become a pro (and even often then) you will almost never have much of an idea about what you should do to answer a particular question, and will instead have to research your options and pick the best one (if one is available at all!).

## Some boilerplate

Let's import a giant pile of stuff! We'll talk about each of these in turn over the course of the presentation.

In [ ]:
from yelp.client import Client
from yelp.oauth1_authenticator import Oauth1Authenticator
from yelp.errors import BusinessUnavailable
import os
import json
from pandas import DataFrame
import folium
import geojson
import random
import requests
import numpy as np
from geopy.distance import vincenty
import bokeh.plotting as plt

Let's authenticate with Yelp!.

Here's the method we will use to import our credentials:

In [ ]:
def import_credentials(filename='yelp_credentials.json'):
    """
    Finds the credentials file describing the token that's needed to access Yelp services.

    :param filename -- The filename at which Yelp service credentials are stored. Defaults to
    `yelp_credentials.json`.
    """
    if filename in [f for f in os.listdir('.') if os.path.isfile(f)]:
        data = json.load(open(filename))
        return data
    else:
        raise IOError('This API requires Yelp credentials to work. Did you forget to define them?')

At this point you need to copy those credentials you have open in another tab (the ones on Yelp! - to reopen them jump to the top of this notebook and read the setup instructions again to get the link).

In [ ]:
auth = Oauth1Authenticator(
    consumer_key='???',
    consumer_secret='???',
    token='???',
    token_secret='???'
)

client = Client(auth)

What is this actually doing? Stuff.

## The core code

**Now here comes the most important bit in this whole notebook. We're going to walk through this method line-by-line**.

In [ ]:
def fetch_businesses(name, area='New York'):
    area = area.lower().replace(' ', '-')
    name = name.lower().replace(' ', '-')
    """
    Fetches all yelp.obj.business_response.BusinessResponse objects for incidences of the given chain in Manhattan.
    Constructs Yelp business ids for incidences of the chain in the area, then queries Yelp to check if they
    exist.
    IDs are constructed name-location-number, so we just have to check numbers in ascending order until it breaks.
    e.g. http://www.yelp.com/biz/gregorys-coffee-new-york-18 is good.
         http://www.yelp.com/biz/gregorys-coffee-new-york-200 is not.
    Then we do reverse GIS searches using the business ID through the Yelp API and extract coordinates from the results.
    Some technical notes:
    1.  The first incidence of any store in the area is reported without any numeral.
        e.g. "dunkin-donuts-new-york", not "dunkin-donuts-new-york-1".
        Numbers pick up from there: the next shitty hole in the wall will be "dunkin-donuts-new-york-2".
    2.  Yelp IDs are unique and are not reassigned when a location is closed.
        Thus we need to check for and exclude closed locations when munging the data.
    3.  Places with a single instance in Manhattan sometimes have a "name-place-2" that redirects to their only location.
        At least this seems to be the case with Bibble & Sip...
        This is checked and corrected for further down the line, by the fetch_businesses() method.
    4.  Sometimes IDs are given to locations that don't actually really exist.
        e.g. the best-buy-3 id points to a non-existant storefront.
        But best-buy-4, best-buy-5, and so on actually exist!
        Yelp acknowledges this fact, but still returns a BusinessUnavaialable error when queries.
        This method sends a web request and checks the response and terminates on a 404, which has proven to be a reliable
        way of circumnavigating this issue.
    5.  In case the above doesn't work...
        The manual_override parameter forces the fetcher to keep moving past this error.
        For debugging purposes, this method prints a URL for the purposes of manually checking breakpoints.
        That way you can incrementally run fetch() and then comb over trouble spots you find by moving up manual_override.
        If you hit that URL and you get either a valid ID or an invalid but existing ID, you need to bump up manual_override
        to correct it and rerun the fetch.
        If you hit that URL and you get a 404 page then you're done!
        e.x. In the Best Buy case both best-buy-3 and best-buy-10 are phantoms.
        But once we set manual_override=10 we're good, and get all of the actual storefronts.
    """
    i = 2
    # Run the first one through by hand.
    try:
        responses = [client.get_business("{0}-{1}".format(name, area))]
    # This can happen, and did, in the Dunkin' Donuts case.
    except BusinessUnavailable:
        responses = []
        pass
    # The rest are handled by a loop.
    while True:
        bus_id = "{0}-{1}-{2}".format(name, area, i)
        try:
            response = client.get_business(bus_id)
        except BusinessUnavailable:
            # We manually check trouble spots.
            # But see the TODO.
            if requests.get('http://www.yelp.com/biz/' + bus_id).status_code != requests.codes.ok:
                break
            else:
                # Increment the counter but don't include the troubled ID.
                i += 1
                continue
        responses += [response]
        i += 1
    print("Ended `fetch_businesses()` on:", "http://www.yelp.com/biz/" + bus_id)
    return responses     

*Long explainy part in words, not in text*

Let's try running this method on an example business. What do we get back?

In [ ]:
bas = fetch_businesses('Bibble and Sip')
bas

We don't need all of the nuts and bolts of the business entity, though all of that is present in these objects. We need just one piece of information: their coordinates. After reading some of the library's documentation (ommitted) you will find that to that you need a somewhat hideously long construction:

In [ ]:
bas[0]

In [ ]:
bas[0].business.location.coordinate.latitude

In [ ]:
[bas[0].business.location.coordinate.latitude, bas[0].business.location.coordinate.longitude]

It works! We can move on to our next chunky method:

In [ ]:
def frame(responses):
    """
    Given a list of yelp.obj.business_response.BusinessResponse objects like the one returns by fetch_businesses(),
    builds a coordinate-logging DataFrame out of them.
    """
    latitudes = [response.business.location.coordinate.latitude for response in responses]
    longitudes = [response.business.location.coordinate.longitude for response in responses]
    df = DataFrame({'latitude': latitudes, 'longitude': longitudes})
    df.index.name=responses[0].business.name
    return df

What do we get?

In [ ]:
frame(bas)

These two locations are in fact one location!

Data science is all about **edge cases**. This problem doesn't apply to chain stores - which is what we will be dealing with - so we can actually, yes, *ignore it*. Wow. Such duct tape.

DataFrames like this one are the interpretive core of the Python data science stack, and you'll get very familiar with them very quickly as you go along.

Next we'll write the Folium method we need to for visualization. Our DataFrame serves as an intermediary!

In [ ]:
def map_coordinates(df):
    """
    Returns a folium map of all of the coordinates stored in a coordinate DataFrame, like the one returned by frame().
    """
    ret = folium.Map(location=[40.753889, -73.983611], zoom_start=11)
    for row in df.iterrows():
        ret.simple_marker([row[1]['latitude'], row[1]['longitude']])
    return ret

In [ ]:
bas_df = frame(bas)
map_coordinates(bas_df)

Let's run this on something a little harder. You'll notice that this query takes a little while to run, because remember that in the background we're sending and reading requests to and from the web: this takes time.

In [ ]:
map_coordinates(frame(fetch_businesses('Gregorys Coffee')))

In [ ]:
starbucks = fetch_businesses('Starbucks')
map_coordinates(frame(starbucks))

## Part II: Measuring average distance

Now that we've got our fancy Starbucks map we can jump into our true end goal: measuring distances. Hold on to your seatbelts, because this is going to be a wild ride!

*Explain methodology and approach*

Ok, let's look at the actual code.

To start with, just take a quick look at [geojson.io](http://geojson.io).

Ok, so let's assume I've generates this shapefile - [here is what my attempt came out to be](https://github.com/ResidentMario/chain-incidence/blob/master/manhattan.geojson). I then saved this file locally - if you followed the instructions at the beginning of this workshop, you will have it!

These first two methods are for loading the shapefile and dumping the result into a list of coordinates.

In [ ]:
def load_geojson(filename="manhattan.geojson"):
    """
    Returns a geojson object for the given file.
    """
    with open(filename) as f:
        dat = f.read()
        obj = geojson.loads(dat)
    return obj


def load_coordinates(name):
    """
    Loads Manhattan.
    What else?
    Are you surprised?
    """
    # Encode according to our storage scheme.
    filename = name.lower().replace(' ', '_') + '.geojson'
    obj = load_geojson(filename)
    if obj['type'] == 'FeatureCollection':
        ret = []
        for feature in obj['features']:
            ret += list(geojson.utils.coords(feature))
        # return ret
    elif obj['type'] == 'Feature':
        ret = list(geojson.utils.coords(obj))
    # GeoJSON stores coordinates [Longitude, Latitude] -- the "modern" format.
    # For historical reasons, coordinates are usually represented in the format [Latitude, Longitude].
    # And this is indeed the format that the rest of the libraries used for this project expect.
    # So we need to swap the two: [Longitude, Latitude] -> [Latitude, Longitude]
    ret = [(coord[1], coord[0]) for coord in ret]
    return ret

Next up, a highly analytical point-in-polygon algorithm. This one we can treat as a black box - I never put in the time to figure out how it works, exactly, and instead just grabbed it off of elsewhere on the Internet.

In [ ]:
def point_inside_polygon(x,y,poly):
    """
    Checks if a point is inside a polygon.
    Used to validate points as being inside of Manahttan.
    Borrowed from: http://www.ariel.com.au/a/python-point-int-poly.html
    
    The shapely library provides features for this and other things besides, but is too much to deal with at the moment.
    """

    n = len(poly)
    inside = False

    p1x,p1y = poly[0]
    for i in range(n+1):
        p2x,p2y = poly[i % n]
        if y > min(p1y,p2y):
            if y <= max(p1y,p2y):
                if x <= max(p1x,p2x):
                    if p1y != p2y:
                        xints = (y-p1y)*(p2x-p1x)/(p2y-p1y)+p1x
                    if p1x == p2x or x <= xints:
                        inside = not inside
        p1x,p1y = p2x,p2y

    return inside

So we have a list of coordinates which are boundaries of Manhattan and a method for checking if some random point is located in that shape. Here are the methods I use for generating a list of points which satisfy our conditions. I'll walk through these bits line-by-line.

In [ ]:
def generate_sample_points(coordinate_list, n=1000):
    """
    Generates n uniformly distributed sample points within the given coordinate list.
    
    When the geometry is sufficiently complex and the list of points large this query can take a while to process.
    """
    lats, longs = list(map(lambda coords: coords[0], coordinate_list)), list(map(lambda coords: coords[1], coordinate_list))
    max_lat = max(lats)
    min_lat = min(lats)
    max_long = max(longs)
    min_long = min(longs)
    ret = []
    while True:
        p_lat = random.uniform(min_lat, max_lat)
        p_long = random.uniform(min_long, max_long)
        if point_inside_polygon(p_lat, p_long, coordinate_list):
            ret.append((p_lat, p_long))
            if len(ret) > n:
                break
        else:
            continue
    return ret


def sample_points(search_location, n=10000):
    """
    Given the name of the location being search, returns n uniformally distributed points within that location.
    
    Wraps the above.
    """
    return generate_sample_points(load_coordinates(search_location), n)

What do we come up with?

In [ ]:
manhattan_point_cloud = sample_points("Manhattan", n=2000)

In [ ]:
plt.output_notebook(hide_banner=True)

p = plt.figure(height=500,
                width=960,
                title="Manhattan Point Cloud",
                x_axis_label="Latitude",
                y_axis_label="Longitude"
               )

p.scatter(
    [coord[0] for coord in manhattan_point_cloud],
    [coord[1] for coord in manhattan_point_cloud]
)

plt.show(p)

Finally, the methods that compress all of this splurge down to just the one number we want: average distance! Again let's walk through this line-by-line.

In [ ]:
def get_minimum_distance(coordinate, coordinate_list):
    """
    Naively calculates the minimum distance in the point cloud.
    """
    best_coord = (0, 0)
    best_distance = 1000
    for candidate_coord in coordinate_list:
        dist = vincenty(coordinate, candidate_coord).miles
        if dist < best_distance:
            best_coord = candidate_coord
            best_distance = dist
    return best_distance


def average_distance(chain_name, search_location, point_cloud):
    """
    This is the main function of this notebook!
    Takes the name of the chain in question and the point cloud associated with the location
    for which we are computing average distance.
    Returns the average distance to that chain within that location.
    We ask for a point cloud and not the name of the location because it's more efficient to precompute an extremely large,
    essentially totally random point cloud, and then check against that, instead of recomputing it every round.
    Output is in feet!
    """
    # First load the coordinates corresponding to the search location..
    location_coords = load_coordinates(search_location)
    # Now generate a list of the chains' locations.
    chain_df = frame(fetch_businesses(chain_name))
    chain_coords = list(zip(chain_df['latitude'], chain_df['longitude']))
    # Finally, get and average the minimum distances between the points in the point cloud and the chain locations.
    distances = [get_minimum_distance(point, chain_coords) for point in point_cloud]
    avg = sum(distances)/len(distances)
    avg_in_feet = int(5280*avg)
    return avg_in_feet

At last we are ready to generate our final answer!

In [ ]:
avg = average_distance("Starbucks", "Manhattan", manhattan_point_cloud)

That concludes this workshop!

# What you need to be doing on your own

Continue working through the starter tutorial list:

 1. [Basic Python Course&mdash;Codeacademy](https://www.codecademy.com/learn/python).
 2. [Crash Course in Numpy and Pandas](https://github.com/anabranch/data_analysis_with_python_and_pandas).
 3. [Data Visualization in Pandas](http://pandas.pydata.org/pandas-docs/stable/visualization.html).
 4. From here you should have enough knowledge to put into action and attempt your first project!

Also: come up with an interesting question that you'd like to try to use data to answer. For inspiration, try leafing through [Dr. Randal Olson's blog](http://www.randalolson.com/blog/) or through [mine](http://www.residentmar.io/). We will start the next meeting off by discussing the problems that you guys come up with, and the techniques that we can use to address them. Who knows - if you come up with something good you can implement it and start off your data science portfolio!


# What we will be doing next time

???